In [ ]:
!pip install playwright selectolax httpx rank-bm25 sentence-transformers torch --upgrade
!playwright install chromium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 59.3 MB/s eta 0:00:00
173.7 MiB [] 0% 132.2s173.7 MiB [] 0% 34.3s173.7 MiB [] 0% 14.2s173.7 MiB [] 0% 9.0s173.7 MiB [] 1% 5.3s173.7 MiB [] 1% 4.2s173.7 MiB [] 2% 3.3s173.7 MiB [] 4% 2.8s173.7 MiB [] 4% 2.6s173.7 MiB [] 5% 2.7s173.7 MiB [] 6% 2.6s173.7 MiB [] 7% 2.4s173.7 MiB [] 8% 2.3s173.7 MiB [] 9% 2.3s173.7 MiB [] 10% 2.2s173.7 MiB [] 12% 2.0s173.7 MiB [] 13% 2.0s173.7 MiB [] 14% 2.0s173.7 MiB [] 15% 1.9s173.7 MiB [] 16% 1.8s173.7 MiB [] 18% 1.7s173.7 MiB [] 19% 1.6s173.7 MiB [] 20% 1.6s173.7 MiB [] 21% 1.6s173.7 MiB [] 22% 1.5s173.7 MiB [] 23% 1.5s173.7 MiB [] 24% 1.5s173.7 MiB [] 25% 1.4s173.7 MiB [] 25% 1.5s173.7 MiB [] 25% 1.6s173.7 MiB [] 26% 1.6s173.7 MiB [] 27% 1.6s173.7 MiB [] 28% 1.5s173.7 MiB [] 29% 1.5s173.7 MiB [] 30% 1.4s173.7 MiB [] 31% 1.4s173.7 MiB [] 32% 1.4s173.7 MiB [] 34% 1.3s173.7 MiB [] 35% 1.3s173.7 MiB [] 36% 1.3s173.7 MiB [] 3

In [ ]:
!pip -q install httpx selectolax

In [ ]:
import json, re, html, sqlite3, asyncio
import httpx
from selectolax.parser import HTMLParser
from pathlib import Path
from datetime import datetime

# --- 0) 입력: 목록 파라미터(list_params)와 쿠키(cookies)를 외부에서 주입 ---
# 예시:
# list_params = {
#   "srchCondition": "...",
#   "srchKeyword": "...",
#   "pageIndex": 1,
#   "pageNo": 1,
#   "curPage": 1,
#   ...  # 실제 스니핑 값 그대로
# }
# cookies = [{"name": "JSESSIONID", "value": "..."}, ...]

TARGET_LIST = "https://portal.scourt.go.kr/pgp/pgp1021/selectLawSrchRsltLst.on"
TARGET_DETAIL = "https://portal.scourt.go.kr/pgp/pgp1025/selectLawDtlList.on"

# --- 1) 목록 페이지 호출 함수 (동일 params로 pageNo만 바꿔가며) ---
async def fetch_rows(page_no:int, cookies:list, list_params:dict):
    if not isinstance(list_params, dict) or "pageNo" not in list_params:
        raise ValueError("list_params(dict)가 필요합니다. (예: {'pageNo':1, 'pageIndex':1, 'curPage':1, ...})")

    payload = {"dma_searchParam": dict(list_params)}
    for k in ("pageNo", "pageIndex", "curPage"):
        if k in payload["dma_searchParam"]:
            payload["dma_searchParam"][k] = page_no

    headers = {
        "Origin": "https://portal.scourt.go.kr",
        "Referer": "https://portal.scourt.go.kr/pgp/index.on?c=900&l=N&m=PGP1021M01",
        "User-Agent": "Mozilla/5.0",
        "Accept": "application/json, text/plain, */*",
        "Content-Type": "application/json;charset=UTF-8",
        "X-Requested-With": "XMLHttpRequest",
    }
    if cookies:
        cookie_str = "; ".join(f"{c['name']}={c['value']}" for c in cookies if c.get("name") and c.get("value"))
        if cookie_str:
            headers["Cookie"] = cookie_str

    async with httpx.AsyncClient(timeout=30.0, headers=headers) as client:
        r = await client.post(TARGET_LIST, json=payload)
        r.raise_for_status()
        js = r.json()
        return js.get("data", {}).get("dlt_lawRslt", []), js.get("token")

In [ ]:
# --- 2) orgdocXmlCtt → 조문 리스트로 정규화 ---
# 파일/노트북 어딘가에 이미 있는 동일 이름 함수를 "이 버전"으로 교체
import re, html, xml.etree.ElementTree as ET

def normalize_articles_from_orgdoc(orgdoc_xml_or_html: str):
    """
    대법원 포털 orgdocXmlCtt가 주는 XML(<법령 …>)을 정확히 파싱해
    <조문>/<조> 단위로 쪼개고, <조문제목>/<조제목>에서 제목·조번호를 추출한다.
    XML이 아니거나 실패 시에는 '제 n 조' 텍스트 분할로 폴백.
    """
    if not orgdoc_xml_or_html:
        return []

    raw = html.unescape(orgdoc_xml_or_html).strip()

    def clean_text(s: str) -> str:
        # 태그 제거 없이 XML 파서가 제공한 text만 다루므로 공백 정리만
        s = re.sub(r"\s+", " ", (s or "")).strip()
        return s

    def node_text(n) -> str:
        # 해당 노드 아래 모든 텍스트를 연결
        return clean_text("".join(n.itertext()))

    def strip_title_from_body(title: str, body: str) -> str:
        # 제목이 본문 첫머리에 중복될 때 1회 제거
        t, b = clean_text(title), clean_text(body)
        if t and b.startswith(t):
            b = b[len(t):].strip()
        return b

    arts = []

    # 1) XML로 파싱 시도
    if raw.startswith("<?xml") or "<법령" in raw:
        try:
            # 일부 응답이 앞뒤에 공백/문자 포함 시 방어
            start = raw.find("<")
            xmltext = raw[start:] if start >= 0 else raw
            root = ET.fromstring(xmltext)

            # (a) 우선 <조문> 단위
            articles_nodes = root.findall(".//조문")
            # (b) 없으면 <조> 단위
            if not articles_nodes:
                articles_nodes = root.findall(".//조")

            for art in articles_nodes:
                # 제목 후보: <조문제목>, <조제목>
                title_node = art.find(".//조문제목")
                if title_node is None:
                    title_node = art.find(".//조제목")
                title = node_text(title_node) if title_node is not None else ""

                # 제목이 없으면 "제 n 조" 패턴을 본문에서 추정
                body_all = node_text(art)
                if not title:
                    m = re.search(r"(제\s*\d+\s*조[^\n]*)", body_all)
                    title = clean_text(m.group(1)) if m else "조문"

                # 조 번호 추출
                mnum = re.search(r"제\s*(\d+)\s*조", title)
                art_no = mnum.group(1) if mnum else None

                # 본문: 해당 조문 노드 전체 텍스트에서 제목 1회 제거
                body = strip_title_from_body(title, body_all)

                # 요약(간단히 ‘다.’ 기준 1문장)
                summary = body.split("다.")[0][:300] if body else ""

                arts.append({
                    "art_no": art_no,
                    "title": title,
                    "body": body,
                    "summary": summary,
                })

            # 부칙 처리(있으면 하나의 항목으로 추가)
            appendix = root.find(".//부칙")
            if appendix is not None:
                btxt = node_text(appendix)
                if btxt:
                    arts.append({"art_no": None, "title": "부칙", "body": btxt, "summary": btxt[:300]})

            if arts:
                return arts

        except ET.ParseError:
            # XML 파싱 실패 → 텍스트 폴백으로
            pass

    # 2) 텍스트/HTML 폴백: "제 n 조" 구획 분할 강화
    flat = clean_text(ET.fromstring(f"<root>{raw}</root>").text or raw) if raw.startswith("<") else clean_text(raw)

    # '제 n 조' 앞에 경계가 오도록 lookahead 사용
    chunks = re.split(r"(?=(?:^|\s)제\s*\d+\s*조\b)", flat)
    for ch in chunks:
        ch = ch.strip()
        if not ch:
            continue
        mt = re.match(r"(제\s*(\d+)\s*조[^\s]*)\s*(.*)", ch)
        if mt:
            title, art_no, body = mt.group(1), mt.group(2), mt.group(3).strip()
            arts.append({
                "art_no": art_no,
                "title": title,
                "body": body,
                "summary": (body.split("다.")[0][:300] if body else "")
            })

    if not arts and flat:
        arts = [{"art_no": None, "title": "본문", "body": flat, "summary": flat[:300]}]
    return arts

In [ ]:
# --- 3) SQLite 스키마 및 적재 ---
DB_PATH = Path("/content/law_catalog.sqlite").as_posix()

def init_db(db_path=DB_PATH):
    con = sqlite3.connect(db_path)
    cur = con.cursor()
    cur.execute("""
        CREATE TABLE IF NOT EXISTS laws (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            official_name TEXT,
            law_type TEXT,
            promulgation_no TEXT,
            promulgation_date TEXT,
            effective_date TEXT,
            crntLawDvsCd TEXT,
            raw_json TEXT
        )
    """)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS articles (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            law_id INTEGER,
            art_no TEXT,
            title TEXT,
            body TEXT,
            summary TEXT,
            FOREIGN KEY(law_id) REFERENCES laws(id)
        )
    """)
    con.commit()
    con.close()

def insert_law_and_articles(row, articles, db_path=DB_PATH):
    key = (row.get("lawHanNm"), row.get("lawPrmlgtNo"), row.get("lawPrmlgtYmd"))
    con = sqlite3.connect(db_path)
    cur = con.cursor()

    # ✅ 같은 법령이면 갱신(UPSERT)
    cur.execute("""
        INSERT INTO laws (official_name, law_type, promulgation_no, promulgation_date, effective_date, crntLawDvsCd, raw_json)
        VALUES (?, ?, ?, ?, ?, ?, ?)
        ON CONFLICT(official_name, promulgation_no, promulgation_date)
        DO UPDATE SET
          law_type      = excluded.law_type,
          effective_date= excluded.effective_date,
          crntLawDvsCd  = excluded.crntLawDvsCd,
          raw_json      = excluded.raw_json
    """, (
        row.get("lawHanNm"),
        row.get("lawCdcsNm"),
        row.get("lawPrmlgtNo"),
        row.get("lawPrmlgtYmd"),
        row.get("lawEnfcYmd"),
        row.get("crntLawDvsCd"),
        json.dumps(row, ensure_ascii=False)
    ))

    # id 확보 (UPSERT 후에도 확실하게)
    cur.execute("""
        SELECT id FROM laws WHERE official_name=? AND promulgation_no=? AND promulgation_date=?
    """, key)
    law_id = cur.fetchone()[0]

    # ✅ 조문도 중복 무시
    cur.executemany("""
        INSERT OR IGNORE INTO articles (law_id, art_no, title, body, summary)
        VALUES (?, ?, ?, ?, ?)
    """, [(law_id, a.get("art_no"), a.get("title"), a.get("body"), a.get("summary")) for a in articles])

    con.commit(); con.close()

async def harvest(pages:int, list_params:dict, cookies:list=None):
    init_db()
    seen = set()  # ✅ 런 중 중복 방지 (법령명, 공포번호, 공포일)
    total_rows = total_articles = 0

    for p in range(1, pages+1):
        rows, _ = await fetch_rows(p, cookies, list_params)
        print(f"[list] page {p}: {len(rows)} rows")
        total_rows += len(rows)

        for r in rows:
            k = (r.get("lawHanNm"), r.get("lawPrmlgtNo"), r.get("lawPrmlgtYmd"))
            if k in seen:
                continue
            seen.add(k)

            text_blob = r.get("orgdocXmlCtt") or ""
            arts = normalize_articles_from_orgdoc(text_blob)
            insert_law_and_articles(r, arts)
            total_articles += len(arts)
    print(f"[done] rows={total_rows}, articles={total_articles}, db={DB_PATH}")

def preview_samples(n=5, db_path=DB_PATH):
    con = sqlite3.connect(db_path)
    cur = con.cursor()
    cur.execute("""
        SELECT l.official_name, a.art_no, a.title, substr(a.body,1,120)
        FROM laws l JOIN articles a ON l.id=a.law_id
        LIMIT ?
    """, (n,))
    rows = cur.fetchall()
    print(rows)
    con.close()

# preview_samples()

In [ ]:
list_params = {
    "aplcnBgngYmd" : "",
    "befSrchwd" : "",
    "bgngYmd" : "",
    "category" : "law",
    "crntLawDvsCd" : "02",
    "crntLawDvsCdGrp" : "",
    "crntLawYn" : "",
    "dtlSrchYn" : "",
    "endYmd" : "",
    "initYn" : "Y",
    "isKwdSearch" : "N",
    "jisCntntsKndCd" : "03|04",
    "lawCdcsCd" : "",
    "lawCdcsCdGrp" : "",
    "lawHanNm" : "",
    "lawPrmlgtNoFrom" : "",
    "lawPrmlgtNoTo" : "",
    "pageNo" : "1",
    "pageSize" : "20",
    "preSrchConditions" : "",
    "prvsNo" : "",
    "reSrchFlag" : "",
    "searchRange" : "entrvsTab",
    "searchScope" : "",
    "sort" : "law_prmlgt_ymd_s desc, law_han_nm_o asc, law_prmlgt_no_s desc, law_enfc_ymd_s desc, law_entrvs_dvs_cd_s asc",
    "sortType" : "최신제개정법령",
    "srchwd" : "",
    "totalCnt" : "",
    "totalCount" : "",
    "totalYn" : "",
    "ymdDvsCd" : ""
}

In [ ]:
# 2) 쿠키 — Request Headers의 cookie: 라인을 아래 헬퍼로 변환
cookie_header = """WMONID=PGgfJVel87r; npPfsHost=127.0.0.1; npPfsPort=14440; JSESSIONID=Vz3hYyY-y_CJ5HrF6uFBcnpNBy0gr9LiAH4pTwL1GwI-UTXBjdbw!789019328; lastAccess=1760422977171"""
def parse_cookie_header(cookie_header: str):
    items = []
    for pair in cookie_header.split(";"):
        if "=" in pair:
            name, value = pair.strip().split("=", 1)
            items.append({"name": name, "value": value})
    return items

cookies = parse_cookie_header(cookie_header)

print(list_params)
print(cookies[:3], "...총", len(cookies), "개")

{'aplcnBgngYmd': '', 'befSrchwd': '', 'bgngYmd': '', 'category': 'law', 'crntLawDvsCd': '02', 'crntLawDvsCdGrp': '', 'crntLawYn': '', 'dtlSrchYn': '', 'endYmd': '', 'initYn': 'Y', 'isKwdSearch': 'N', 'jisCntntsKndCd': '03|04', 'lawCdcsCd': '', 'lawCdcsCdGrp': '', 'lawHanNm': '', 'lawPrmlgtNoFrom': '', 'lawPrmlgtNoTo': '', 'pageNo': '1', 'pageSize': '20', 'preSrchConditions': '', 'prvsNo': '', 'reSrchFlag': '', 'searchRange': 'entrvsTab', 'searchScope': '', 'sort': 'law_prmlgt_ymd_s desc, law_han_nm_o asc, law_prmlgt_no_s desc, law_enfc_ymd_s desc, law_entrvs_dvs_cd_s asc', 'sortType': '최신제개정법령', 'srchwd': '', 'totalCnt': '', 'totalCount': '', 'totalYn': '', 'ymdDvsCd': ''}
[{'name': 'WMONID', 'value': 'PGgfJVel87r'}, {'name': 'npPfsHost', 'value': '127.0.0.1'}, {'name': 'npPfsPort', 'value': '14440'}] ...총 5 개


In [49]:
import nest_asyncio, asyncio
nest_asyncio.apply()

# 1) 페이지 수만큼 적재
await harvest(pages=11, list_params=list_params, cookies=cookies)

# 2) 결과 미리보기
preview_samples(5)

[list] page 1: 20 rows
[list] page 2: 20 rows
[list] page 3: 20 rows
[list] page 4: 20 rows
[list] page 5: 20 rows
[list] page 6: 20 rows
[list] page 7: 20 rows
[list] page 8: 20 rows
[list] page 9: 20 rows
[list] page 10: 20 rows
[list] page 11: 2 rows
[done] rows=202, articles=0, db=law_catalog.sqlite
[]


In [50]:
import os, sqlite3

DB_PATH = r"law_catalog.sqlite"  # 너가 실제로 여는 파일 경로로!
print("DB_PATH:", DB_PATH, "exists:", os.path.exists(DB_PATH), "size:", os.path.getsize(DB_PATH) if os.path.exists(DB_PATH) else -1)

with sqlite3.connect(DB_PATH) as con:
    cur = con.cursor()
    cur.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name")
    print("tables:", [r[0] for r in cur.fetchall()])


DB_PATH: law_catalog.sqlite exists: True size: 733184
tables: ['articles', 'laws', 'sqlite_sequence']


In [51]:
import sqlite3, json

with sqlite3.connect(DB_PATH, timeout=30) as con:
    con.execute("PRAGMA journal_mode=WAL")
    con.execute("PRAGMA busy_timeout=10000")
    con.execute("PRAGMA synchronous=NORMAL")
    cur = con.cursor()

    # laws가 비어있으면 여기서 중단
    cur.execute("SELECT COUNT(*) FROM laws")
    if cur.fetchone()[0] == 0:
        raise RuntimeError("laws 테이블이 비어 있습니다. 먼저 harvest로 laws를 채우세요.")

    cur.execute("BEGIN IMMEDIATE")
    cur.execute("DELETE FROM articles")

    cur.execute("SELECT id, raw_json FROM laws")
    rows = cur.fetchall()

    insert_sql = "INSERT INTO articles (law_id, art_no, title, body, summary) VALUES (?, ?, ?, ?, ?)"
    total = 0
    for law_id, raw in rows:
        row = json.loads(raw)
        arts = normalize_articles_from_orgdoc(row.get("orgdocXmlCtt", "") or "")
        cur.executemany(insert_sql, [
            (law_id, a.get("art_no"), a.get("title"), a.get("body"), a.get("summary")) for a in arts
        ])
        total += len(arts)

    con.commit()

print("articles repopulated:", total)


articles repopulated: 0
